In [49]:
                                             **IMPLEMENTATION OF GRU ON TIME SERIES DATA**   
# Import necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping


In [33]:
# Load test features and labels
X_test = np.load('rolling_window_sequences.npy')  # Replace with actual file path
metadata_test = pd.read_csv("sequence_metadata_with_RUL.csv")  # Replace with actual file path
y_test = metadata_test["RUL"].values
print("Test feature shape:", X_test.shape)
print("Test target shape:", y_test.shape)

Test feature shape: (17731, 30, 66)
Test target shape: (17631,)


In [39]:
# Split the data into train and validation sets (assuming no separate train set given)
X_train, X_val, y_train, y_val = train_test_split(X_test, y_test, test_size=0.3, random_state=42)


In [40]:
for i in range(X_train.shape[2]):
    mean_val = np.nanmean(X_train[:,:,i])
    X_train[:,:,i][np.isnan(X_train[:,:,i])] = mean_val

for i in range(X_val.shape[2]):
    mean_val = np.nanmean(X_val[:,:,i])
    X_val[:,:,i][np.isnan(X_val[:,:,i])] = mean_val




In [41]:
from sklearn.preprocessing import MinMaxScaler

n_samples, n_timesteps, n_features = X_train.shape
X_train_2d = X_train.reshape(-1, n_features)
scaler_X = MinMaxScaler()
X_train_scaled = scaler_X.fit_transform(X_train_2d)
X_train = X_train_scaled.reshape(n_samples, n_timesteps, n_features)

# Apply same scaler to validation set
X_val = scaler_X.transform(X_val.reshape(-1, n_features)).reshape(X_val.shape)


In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, Input

def create_gru_model(input_shape, units=64, dropout_rate=0.2):
    model = Sequential()
    model.add(Input(shape=input_shape))  # input layer
    model.add(GRU(units, return_sequences=False))  # GRU layer
    model.add(Dropout(dropout_rate))  # dropout for regularization
    model.add(Dense(1, activation='linear'))  # output layer
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

model = create_gru_model(input_shape=(X_train.shape[1], X_train.shape[2]))


In [43]:
from tensorflow.keras.callbacks import EarlyStopping

callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
]


In [44]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)


Epoch 1/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 12s 44ms/step - loss: 6661.9150 - mae: 70.1395 - val_loss: 5643.8394 - val_mae: 63.8439
Epoch 2/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 8s 44ms/step - loss: 4996.0312 - mae: 59.8251 - val_loss: 4380.2280 - val_mae: 55.8751
Epoch 3/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - loss: 3921.5156 - mae: 53.0259 - val_loss: 3477.5120 - val_mae: 50.0292
Epoch 4/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 8s 40ms/step - loss: 3166.5781 - mae: 48.1469 - val_loss: 2855.8608 - val_mae: 45.9568
Epoch 5/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 11s 54ms/step - loss: 2656.1814 - mae: 44.6824 - val_loss: 2442.6714 - val_mae: 43.1489
Epoch 6/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 10s 49ms/step - loss: 2328.3926 - mae: 42.3810 - val_loss: 2184.6155 - val_mae: 41.2750
Epoch 7/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 11s 55ms/step - loss: 2127.5896 - mae: 40.8349 - val_loss: 2031.2280 - val_mae: 40.0838
Epoch 8/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 10s 52ms/step - loss: 2006.6455 - mae: 39.8111 - val_loss: 1947.127

In [45]:
val_loss, val_mae = model.evaluate(X_val, y_val, verbose=0)
print(f'Validation MSE: {val_loss:.4f}, MAE: {val_mae:.4f}')


Validation MSE: 1872.8556, MAE: 38.4533


In [48]:
y_pred = model.predict(X_test).flatten()


551/551 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step
